# Tables
- Number of all and valid gages: US, HUC2, aquifer
- Distribution of HMF metrics: US, HUC2, aquifer
- Gages with the highest annual volume

In [1]:
# IMPORTS
import os
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
#import contextily as cx
import requests
import calendar
from importlib import reload

from datetime import datetime, timedelta
from shapely.geometry import Point
from io import StringIO
from mpl_toolkits.axes_grid1 import make_axes_locatable

# USGS Data retreival tool
from dataretrieval import nwis, utils, codes

# Custom modules are imported in multiple locations to faciliate easy reloading when edits are made to their respective files
import Src.classes as cl
import Src.func_ko as fn
reload(cl)
reload(fn)

<module 'Src.func_ko' from 'C:\\Users\\kondris\\Documents\\GitHub\\HighMagFlows_EPA_Project\\Src\\func_ko.py'>

In [2]:
km3_to_maf = (1000)**3 * (3.28)**3 / (43560) / 1000000
1*km3_to_maf

0.8100907254361798

# Import national metrics

In [3]:
# All gages - National metrics dfs
# data_paths = {
#     '30_90': 'Prelim_Data/National_Metrics/National_Metrics_30_90.xlsx',
#     '50_90': 'Prelim_Data/National_Metrics/National_Metrics_50_90.xlsx',
#     '30_95': 'Prelim_Data/National_Metrics/National_Metrics_30_95.xlsx',
#     '50_95': 'Prelim_Data/National_Metrics/National_Metrics_50_95.xlsx'    
# }

#dfs_metrics = {key: pd.read_excel(path, sheet_name='site_metrics') for key, path in data_paths.items()}

data_paths = {
    '30_90': 'Prelim_Data/National_Metrics/Station_names/National_Metrics_30_90.xlsx',
    '50_90': 'Prelim_Data/National_Metrics/Station_names/National_Metrics_50_90.xlsx',
    '30_95': 'Prelim_Data/National_Metrics/Station_names/National_Metrics_30_95.xlsx',
    '50_95': 'Prelim_Data/National_Metrics/Station_names/National_Metrics_50_95.xlsx'    
}

dfs_valid = {key: pd.read_excel(path) for key, path in data_paths.items()}

In [4]:
# # Converts site_no to strings
# date_ranges = ['30', '50']
# percentiles = ['90', '95']
# for date_range in date_ranges:
#     for percentile in percentiles: 
#         # Assuming df is your DataFrame and 'column_name' is the name of the column with numbers
#         dfs_metrics[f'{date_range}_{percentile}']['site_no'] = dfs_metrics[f'{date_range}_{percentile}']['site_no'].astype(str)  # Convert numbers to strings

#         # Add leading '0' to numbers with 7 digits
#         dfs_metrics[f'{date_range}_{percentile}']['site_no'] = dfs_metrics[f'{date_range}_{percentile}']['site_no'].apply(lambda x: '0' + x if len(x) == 7 else x)

# Converts site_no to strings
date_ranges = ['30', '50']
percentiles = ['90', '95']
for date_range in date_ranges:
    for percentile in percentiles: 
        # Assuming df is your DataFrame and 'column_name' is the name of the column with numbers
        dfs_valid[f'{date_range}_{percentile}']['site_no'] = dfs_valid[f'{date_range}_{percentile}']['site_no'].astype(str)  # Convert numbers to strings

        # Add leading '0' to numbers with 7 digits
        dfs_valid[f'{date_range}_{percentile}']['site_no'] = dfs_valid[f'{date_range}_{percentile}']['site_no'].apply(lambda x: '0' + x if len(x) == 7 else x)

In [5]:
# Valid gages - National metrics dfs
# dfs_valid = {}
# for date_range in date_ranges:
#     for percentile in percentiles: 
#         dfs_valid[f'{date_range}_{percentile}'] = dfs_metrics[f'{date_range}_{percentile}'][dfs_metrics[f'{date_range}_{percentile}']['valid'] == True]

## Import national metrics with additional columns

In [105]:
# All gages - National metrics dfs
data_paths = {
    '30_90': 'Tables/valid_gages_30_90.xlsx',
    '50_90': 'Tables/valid_gages_50_90.xlsx',
    '30_95': 'Tables/valid_gages_30_95.xlsx',
    '50_95': 'Tables/valid_gages_50_95.xlsx'    
}

dfs_valid_within_aq = {key: pd.read_excel(path) for key, path in data_paths.items()}

In [108]:
dfs_valid_within_aq['30_90'].shape

(4242, 46)

## Import national metrics with new metrics (percent HMF)

In [7]:
data_paths = {
    '30_90': 'Prelim_Data/National_Metrics/National_Metrics_update_30_90.xlsx',
    '50_90': 'Prelim_Data/National_Metrics/National_Metrics_update_50_90.xlsx',
    '30_95': 'Prelim_Data/National_Metrics/National_Metrics_update_30_95.xlsx',
    '50_95': 'Prelim_Data/National_Metrics/National_Metrics_update_50_95.xlsx'    
}

dfs_valid_update = {key: pd.read_excel(path) for key, path in data_paths.items()}

In [8]:
# Converts site_no to strings
date_ranges = ['30', '50']
percentiles = ['90', '95']
for date_range in date_ranges:
    for percentile in percentiles: 
        # Assuming df is your DataFrame and 'column_name' is the name of the column with numbers
        dfs_valid_update[f'{date_range}_{percentile}']['site_no'] = dfs_valid_update[f'{date_range}_{percentile}']['site_no'].astype(str)  # Convert numbers to strings

        # Add leading '0' to numbers with 7 digits
        dfs_valid_update[f'{date_range}_{percentile}']['site_no'] = dfs_valid_update[f'{date_range}_{percentile}']['site_no'].apply(lambda x: '0' + x if len(x) == 7 else x)

# Create new regions

In [5]:
date_ranges = ['30', '50']
percentiles = ['90', '95']

for date_range in date_ranges:
    for percentile in percentiles: 
        df = dfs_valid[f'{date_range}_{percentile}']
        #df = dfs_valid_update[f'{date_range}_{percentile}']
        
        df_huc2_04 = df[df['huc2_code'] == 4]
        unique_huc4_huc2_04 = df_huc2_04['huc4_code'].unique().tolist()
        huc2_04_northeast_list = [413, 414, 415]
        huc2_04_midwest_list = [huc for huc in unique_huc4_huc2_04 if huc not in huc2_04_northeast_list]
        
        df_huc2_18 = df[df['huc2_code'] == 18]
        unique_huc4_huc2_18 = df_huc2_18['huc4_code'].unique().tolist()
        huc2_04_southwest_list = [huc for huc in unique_huc4_huc2_18 if huc != 1801]
        
        df['region'] = 'NA'
        
        # Pacific Northwest
        df.loc[(df['huc2_code'] == 17) | (df['huc4_code'] == 1801), 'region'] = 'pac_northwest'
        
        # Upper Plains
        df.loc[df['huc2_code'].isin([9, 10]), 'region'] = 'upper_plains'
        
        # Midwest
        df.loc[(df['huc2_code'].isin([5, 7])) | (df['huc4_code'].isin(huc2_04_midwest_list)), 'region'] = 'midwest'
        
        # Northeast
        df.loc[(df['huc2_code'].isin([1, 2])) | (df['huc4_code'].isin(huc2_04_northeast_list)), 'region'] = 'northeast'
        
        # Southwest
        df.loc[(df['huc2_code'].isin([14, 15, 16])) | (df['huc4_code'].isin(huc2_04_southwest_list)), 'region'] = 'southwest'
        
        # South Central
        df.loc[df['huc2_code'].isin([11, 12, 13]), 'region'] = 'southcentral'
        
        # Southeast
        df.loc[df['huc2_code'].isin([3, 6, 8]), 'region'] = 'southeast'

        dfs_valid[f'{date_range}_{percentile}'] = df
        #dfs_valid_update[f'{date_range}_{percentile}'] = df

In [6]:
#dfs_valid_update['30_90'].head()
dfs_valid['30_90'].head()

,Unnamed: 0,site_no,analyze_start,analyze_end,analyze_range,quantile,valid,missing_data%,threshold,hmf_years,...,data_start,data_end,total_record,state,huc2_code,huc4_code,within_aq,HCDN_2009,station_nm,region
0,1,02342500,1990-10-01,2020-09-30,30,0.9,True,0.0,850.3,30,...,1946-10-01,2020-09-30,74.0,AL,3,313,Southeastern Coastal Plain aquifer system,False,"UCHEE CREEK NEAR FORT MITCHELL, AL.",southeast
1,2,02361000,1990-10-01,2020-09-30,30,0.9,True,0.0,1770.0,30,...,1921-12-01,2020-09-30,98.8,AL,3,314,Southeastern Coastal Plain aquifer system,False,"CHOCTAWHATCHEE RIVER NEAR NEWTON, AL.",southeast
2,4,02363000,1990-10-01,2020-09-30,30,0.9,True,0.0,1300.0,30,...,1938-10-01,2020-09-30,82.0,AL,3,314,Southeastern Coastal Plain aquifer system,False,PEA RIVER NEAR ARITON AL,southeast
3,6,02369800,1990-10-01,2020-09-30,30,0.9,True,0.0,261.0,29,...,1967-10-01,2020-09-30,53.0,AL,3,314,Coastal lowlands aquifer system,False,BLACKWATER RIVER NEAR BRADLEY AL,southeast
4,7,02371500,1990-10-01,2020-09-30,30,0.9,True,0.0,1380.0,30,...,1937-10-01,2020-09-30,83.0,AL,3,314,Southeastern Coastal Plain aquifer system,False,CONECUH RIVER AT BRANTLEY AL,southeast


# Misc. functions

## Convert DOHY to date

In [7]:
from datetime import datetime, timedelta

def convert_day_of_year(day_number, base_year=2023):
    # Calculate the start date (October 1 of the base year)
    start_date = pd.to_datetime(f"{base_year}-10-01")
    
    # Adjust the day number by subtracting 1 to account for the start date
    adjusted_day_number = day_number - 0
    
    # Calculate the final date by adding the adjusted day number to the start date
    final_date = start_date + pd.DateOffset(days=adjusted_day_number)
    
    return final_date

convert_day_of_year(248)

# for date_range in date_ranges:
#     for percentile in percentiles: 
#         dfs_valid[f'{date_range}_{percentile}']['timing'] = pd.to_numeric(dfs_valid[f'{date_range}_{percentile}']['timing'])
#         dfs_valid[f'{date_range}_{percentile}']['timing'] = dfs_valid[f'{date_range}_{percentile}']['timing'].round()
#         dfs_valid[f'{date_range}_{percentile}']['com_date'] = convert_day_of_year(dfs_valid[f'{date_range}_{percentile}']['timing'].apply(convert_day_of_year))

Timestamp('2024-06-05 00:00:00')

## Determine highest gages of a certain metric

In [ ]:
date_range = '30'
percentile = '90'
metric = 'annual_hmf'
df = dfs_valid[f'{date_range}_{percentile}']
df = df.dropna(subset='within_aq')
df = df.sort_values(by=[metric], ascending=False)
df[[metric, 'station_nm']][0:10]

# Add updated within_aq column for selected aquifers

In [101]:
gdf_selected_aq = gpd.read_file('ShapeFiles/Aquifers_select/selected_aquifers.shp')

for date_range in date_ranges:
    for percentile in percentiles: 
        df = dfs_valid[f'{date_range}_{percentile}'].copy()
        
        geo_df = fn.convert_geometry(df)
        geo_df = geo_df.merge(df, how='left', left_index=True, right_index=True)
        geo_df.crs = epsg=4269
        geo_df = geo_df.to_crs(epsg=4269)

        gdf_selected_aq = gdf_selected_aq.to_crs(epsg=4269)

        df_upd = gpd.sjoin(geo_df, gdf_selected_aq[['AQ_NAME', 'geometry']], how="left", predicate="within")
        df_upd = df_upd.rename(columns={"AQ_NAME": "within_aq_selected"})
        dfs_valid[f'{date_range}_{percentile}'].loc[:, 'within_aq_selected'] = df_upd['within_aq_selected'].values

ValueError: Must have equal len keys and value when setting with an ndarray

In [93]:
#gdf_aq_selected

In [100]:
dfs_valid['30_90']['within_aq_selected'].unique()

array([nan], dtype=object)

# Save HMF metrics for all gages

## Outlet gages

In [82]:
# NEW huc4 outlet gages by aquifer
hp_outlet_gages = ['06447000', '06465500', '06674500', '06764880', '06801000', '06793000', '06800500', '06853500', '06873000', '06881000', 
                   '07134180', '07144300', '07157500', '07227500', '07235000', '07297910']
mr_outlet_gages = ['07022000', '07077000', '07288280', '07364200', '07369000', '07074500']
cv_outlet_gages = ['11447650', '11200800', '11303500']
br_outlet_gages = ['09415000', '09521100', '09466500', '09469500', '09502000', '09519800', '09537200', '10126000', '10141000', '10219000', 
                   '10327500', '10351700', '10249280', '13105000', '11062000', '10260500', '10254730']
fl_outlet_gages = ['02240000', '02313000', '02320500', '02326900', '02358000', '02365500']
sr_outlet_gages = ['13087995', '13269000']
cl_outlet_gages = ['02375500', '02471001', '02479000', '02489500', '07373000', '07290000', '07378500', '08013500', '07375000', '08030500', 
                   '08033500', '08066500', '08068000', '08116650', '08162000', '08176500', '08211000']
cc_outlet_gages = ['11477000', '11451000', '11458000', '11152500', '11078000']
pn_outlet_gages = ['10039500', '10172952', '10346000', '12398600', '12447200', '13087995', '13269000', '13317000', '14038530', '14233500', 
                   '14187200', '14321000', '12045500', '10396000', '11509500', '11377100']
na_outlet_gages = ['01402000', '01463500', '01578310', '01594440', '01646500', '02037500', '02080500', '02083500', '02105500', '02129000']
cp_outlet_gages = ['12422500', '12472800', '12510500', '13334300', '14105700']
rg_outlet_gages = ['08317400', '08361000']
me_outlet_gages = ['02428400', '02469761', '02477000', '02482550', '03611260', '03610200', '07029500', '07047942', '07268000', '07362000', 
                   '07289350', '07337000']
co_outlet_gages = ['04024000', '04045500', '04082400', '05331000', '05330000', '05340500', '05378500', '05369500', '05388250', '05404000', 
                   '05437500', '05514500', '05552500', '05555300']
sc_outlet_gages = ['02130561', '02169500', '02197000', '02223500', '02343801', '02373000', '02425000', '02467000', '03399800', '03610200']
bi_outlet_gages = ['02289060']
et_outlet_gages = ['07316000', '07335300', '08044500', '08091000', '08102500', '08123850', '08158000', '08167800', '08190000', '08446500']
rm_outlet_gages = ['06036650', '06054500', '06192500', '12395500', '13142500', '13247500', '13302500']
pb_outlet_gages = ['01388500', '01463500', '01578310', '01594000', '01646500', '02037500', '02066000', '02082585', '02102500', '02129000', 
                   '02169500', '02192000', '02213000', '02341460', '02414715', '03168000', '03453500', '03550000']
sa_outlet_gages = ['02105500', '02131000', '02171500', '02198500', '02226000', '02244040', '02292900', '02296750', '02315500', '02330000', 
                   '02359170', '02366500']
vr_outlet_gages = ['01446500', '01570500', '01613000', '02019500', '02056000', '02397000', '02455000', '03176500', '03540500', '03568933']
sd_outlet_gages = ['04056500', '04087000', '04126740', '04127997', '04166500', '04193500', '04127917', '03227500', '03270500', '03237500', 
                   '03283500', '03335500', '03434500', '03294500', '03604000', '05412500', '05420500', '05446500', '05587450', '05527500', 
                   '05570910']
pnb_outlet_gages = ['06037500', '06188000', '10352500', '12414500', '12488500', '13062500', '13269000', '13341050', '14046500', '14246900', 
                    '14211720', '14301000', '12205000', '10396000', '11509500', '11377100']
wl_outlet_gages = ['14142500', '14211720']
lc_outlet_gages = ['05270700', '05311000', '05449500', '05476750', '05082500', '06090300', '06208500', '06287000', '06436000', '06479010', 
                   '06630000', '06801000', '06800500', '06486000', '06809500', '06856600', '06876900', '06884025', '07140850']
uc_outlet_gages = ['08284100', '09163500', '09152500', '09180500', '09217000', '09260050', '09315000', '09380000', '09355500', '09408150', 
                   '09402000', '09504420', '10020100', '10155000', '10217000', '13018300']

outlet_gages_dict_selected = {
    'pb': pb_outlet_gages,
    'br_az': br_az_outlet_gages,
    'cv_sac': cv_sac_outlet_gages,
    'cv_sj': cv_sj_outlet_gages,
    'cl_cl': cl_cl_outlet_gages,  
    'cp': cp_outlet_gages,
    'et': et_outlet_gages,
    'fl': fl_outlet_gages,
    'hp_n': hp_n_outlet_gages,
    'hp_c_s': hp_c_s_outlet_gages,    
    'mr': mr_outlet_gages,
    'me': me_outlet_gages,
    'sr': sr_outlet_gages,
    'cl_tx': cl_tx_outlet_gages,
    'mr_me':  mr_outlet_gages + me_outlet_gages,
    'uc': uc_outlet_gages
}

outlet_gages_dict_25 = {
    'br': br_outlet_gages, 
    'cc': cc_outlet_gages,
    'cv': cv_outlet_gages,
    'cl': cl_outlet_gages,
    'fl': fl_outlet_gages,
    'hp': hp_outlet_gages,
    'mr': mr_outlet_gages,
    'na': na_outlet_gages,
    'pn': pn_outlet_gages,
    'sr': sr_outlet_gages,
    'cp': cp_outlet_gages,
    'rg': rg_outlet_gages,
    'me': me_outlet_gages,
    'co': co_outlet_gages,
    'sc': sc_outlet_gages,
    'bi': bi_outlet_gages,
    'et': et_outlet_gages,
    'rm': rm_outlet_gages,
    'pb': pb_outlet_gages,
    'sa': sa_outlet_gages,
    'vr': vr_outlet_gages,
    'sd': sd_outlet_gages,
    'pnb': pnb_outlet_gages,
    'wl': wl_outlet_gages,
    'lc': lc_outlet_gages,
    'hp_n': hp_n_outlet_gages,
    'hp_c_s': hp_c_s_outlet_gages,
    'cv_sac': cv_sac_outlet_gages,
    'cv_sj': cv_sj_outlet_gages,
    'cl_tx': cl_tx_outlet_gages,
    'cl_cl': cl_cl_outlet_gages,
    'br_az': br_az_outlet_gages,
    'mr_me':  mr_outlet_gages + me_outlet_gages,
    'uc': uc_outlet_gages
}

aq_names_selected = {
                'mr': 'Mississippi River Valley alluvial aquifer',
                'fl': 'Floridan aquifer system',
                'sr': 'Snake River Plain basaltic-rock aquifers',
                'hp_n': 'Northern High Plains',
                'hp_c_s': 'Central-South High Plains',
                'cv_sac': 'Sacramento River Basin',
                'cv_sj': 'San Joaquin and Tulare River Basin',
                'cl_tx': 'Texas Gulf Coast',
                'cl_cl': 'Coastal Lowlands',
                'br_az': 'Arizona Alluvial',
                'me': 'Mississippi embayment aquifer system',
                'et': 'Edwards-Trinity aquifer system',
                'cp': 'Columbia Plateau basaltic-rock aquifers',
                'pb': 'Piedmont and Blue Ridge aquifers',
                'mr_me':  'Mississippi Embayment',
                'uc': 'Upper Colorado'
              }

aq_names_25 = {'hp': 'High Plains aquifer',
                'mr': 'Mississippi River Valley alluvial aquifer',
                'cv': 'Central Valley aquifer system',
                'br': 'Basin and Range basin-fill aquifers',
                'fl': 'Floridan aquifer system',
                'sr': 'Snake River Plain basaltic-rock aquifers',
                'cl': 'Coastal lowlands aquifer system',
                'cc': 'California Coastal Basin aquifers', 
                'pn': 'Pacific Northwest basin-fill aquifers',
                'na': 'Northern Atlantic Coastal Plain aquifer system',
                'cp': 'Columbia Plateau basaltic-rock aquifers',
                'rg': 'Rio Grande aquifer system',
                'me': 'Mississippi embayment aquifer system',
                'co': 'Cambrian-Ordovician aquifer system',
                'sc': 'Southeastern Coastal Plain aquifer system',
                'bi': 'Biscayne aquifer',
                'et': 'Edwards-Trinity aquifer system',
                'rm': 'Northern Rocky Mountains Intermontane Basins aquifer system',
                'pb': 'Piedmont and Blue Ridge aquifers',
                'sa': 'Surficial aquifer system',
                'vr': 'Valley and Ridge aquifers',
                'sd': 'Silurian-Devonian aquifers',
                'pnb': 'Pacific Northwest basaltic-rock aquifers',
                'wl': 'Willamette Lowland basin-fill aquifers',
                'lc': 'Lower Cretaceous aquifers',
                'hp_n': 'Northern High Plaisn',
                'hp_c_s': 'Central-South High Plains',
                'cv_sac': 'Sacramento River Basin',
                'cv_sj': 'San Joaquin and Tulare River Basin',
                'cl_tx': 'Texas Gulf Coast',
                'cl_cl': 'Coastal Lowlands',
               'br_az': 'Arizona Alluvial',
               'mr_me':  'Mississippi Embayment',
               'uc': 'Upper Colorado'
              }

aq_codes_10 = ['hp', 'mr', 'cv', 'br', 'fl', 'sr', 'cl', 'cc', 'pn', 'na']

aq_codes_selected = ['mr', 'fl', 'sr', 'hp_n', 'hp_c_s', 'cv_sac', 'cv_sj', 
                     'cl_tx', 'cl_cl', 'br_az', 'me', 'et',  'cp', 'pb', 'mr_me', 'uc']

aq_codes_25 = ['hp', 'mr', 'cv', 'br', 'fl', 'sr', 'cl', 'cc', 'pn', 'na',
              'cp', 'rg', 'me', 'co', 'sc', 'bi', 'et', 'rm', 'pb', 'sa',
              'vr', 'sd', 'pnb', 'wl', 'lc', 'hp_n', 'hp_c_s', 'cv_sac', 'cv_sj',
              'cl_tx', 'cl_cl', 'br_az', 'mr_me', 'uc']

aq_names_25_list = ['Southeastern Coastal Plain aquifer system',
       'Coastal lowlands aquifer system', 'Floridan aquifer system',
       'Valley and Ridge aquifers',
       'Piedmont and Blue Ridge crystalline-rock aquifers',
       'Mississippi embayment aquifer system',
       'Basin and Range basin-fill aquifers',
       'Mississippi River Valley alluvial aquifer',
       'Pacific Northwest basin-fill aquifers',
       'California Coastal Basin aquifers',
       'Central Valley aquifer system',
       'Pacific Northwest basaltic-rock aquifers',
       'Northern Atlantic Coastal Plain aquifer system',
       'Surficial aquifer system', 'Biscayne aquifer',
       'Northern Rocky Mountains Intermontane Basins aquifer system',
       'Snake River Plain basaltic-rock aquifers',
       'Columbia Plateau basaltic-rock aquifers',
       'Cambrian-Ordovician aquifer system', 'Silurian-Devonian aquifers',
       'Lower Cretaceous aquifers', 'High Plains aquifer',
       'Rio Grande aquifer system', 'Edwards-Trinity aquifer system',
       'Willamette Lowland basin-fill aquifers',
        'Northern High Plaisn',
        'Central-South High Plains',
        'Sacramento River Basin',
         'San Joaquin and Tulare River Basin',
         'Texas Gulf Coast',
          'Coastal Lowlands',
          'Arizona Alluvial',
          'Mississippi Embayment',
          'Upper Colorado']

aq_names_10_list = {'High Plains aquifer',
                'Mississippi River Valley alluvial aquifer',
                'Central Valley aquifer system',
                'Basin and Range basin-fill aquifers',
                'Floridan aquifer system',
                'Snake River Plain basaltic-rock aquifers',
                'Coastal lowlands aquifer system',
                'California Coastal Basin aquifers', 
                'Pacific Northwest basin-fill aquifers',
                'Northern Atlantic Coastal Plain aquifer system'}

aq_names_selected_list = {
                'Mississippi River Valley alluvial aquifer',
                'Floridan aquifer system',
                'Snake River Plain basaltic-rock aquifers',
                'Northern High Plains',
                'Central-South High Plains',
                'Sacramento River Basin',
                'San Joaquin and Tulare River Basin',
                'Texas Gulf Coast',
                'Coastal Lowlands',
                'Arizona Alluvial',
                'Mississippi embayment aquifer system',
                'Edwards-Trinity aquifer system',
                'Columbia Plateau basaltic-rock aquifers',
                'Piedmont and Blue Ridge aquifers',
                'Mississippi Embayment',
                'Upper Colorado'
              }

## Save master df with outlet gages

In [77]:
def assign_region(site_no):
    for aq, gages in outlet_gages_dict_25.items():
        if str(site_no) in gages:  # Ensure site_no is checked as string
            return aq_names_25[aq] # aq_names_25 is another dictionary with the full aquifer name for the identifier
    return ' '

for date_range in date_ranges:
    for percentile in percentiles:
        df = dfs_valid[f'{date_range}_{percentile}'] 
        df['outlet_gage'] = df['site_no'].apply(assign_region)
        file_name = f'valid_gages_{date_range}_{percentile}.xlsx'
        df.to_excel('Tables/'+file_name)

In [78]:
dfs_valid['30_90'].head(10)

,Unnamed: 0,site_no,analyze_start,analyze_end,analyze_range,quantile,valid,missing_data%,threshold,hmf_years,...,data_end,total_record,state,huc2_code,huc4_code,within_aq,HCDN_2009,station_nm,region,outlet_gage
0,1,02342500,1990-10-01,2020-09-30,30,0.9,True,0.000,850.3,30,...,2020-09-30,74.0,AL,3,313,Southeastern Coastal Plain aquifer system,False,"UCHEE CREEK NEAR FORT MITCHELL, AL.",southeast,
1,2,02361000,1990-10-01,2020-09-30,30,0.9,True,0.000,1770.0,30,...,2020-09-30,98.8,AL,3,314,Southeastern Coastal Plain aquifer system,False,"CHOCTAWHATCHEE RIVER NEAR NEWTON, AL.",southeast,
2,4,02363000,1990-10-01,2020-09-30,30,0.9,True,0.000,1300.0,30,...,2020-09-30,82.0,AL,3,314,Southeastern Coastal Plain aquifer system,False,PEA RIVER NEAR ARITON AL,southeast,
3,6,02369800,1990-10-01,2020-09-30,30,0.9,True,0.000,261.0,29,...,2020-09-30,53.0,AL,3,314,Coastal lowlands aquifer system,False,BLACKWATER RIVER NEAR BRADLEY AL,southeast,
4,7,02371500,1990-10-01,2020-09-30,30,0.9,True,0.000,1380.0,30,...,2020-09-30,83.0,AL,3,314,Southeastern Coastal Plain aquifer system,False,CONECUH RIVER AT BRANTLEY AL,southeast,
5,8,02372250,1990-10-01,2020-09-30,30,0.9,True,0.000,1230.0,29,...,2020-09-30,46.0,AL,3,314,Southeastern Coastal Plain aquifer system,False,PATSALIGA CREEK NEAR BRANTLEY AL,southeast,
6,11,02373000,1990-10-01,2020-09-30,30,0.9,True,0.146,1670.0,29,...,2020-09-30,83.0,AL,3,314,Southeastern Coastal Plain aquifer system,False,SEPULGA RIVER NEAR MCKENZIE AL,southeast,Southeastern Coastal Plain aquifer system
7,13,02374500,1990-10-01,2020-09-30,30,0.9,True,0.000,482.0,29,...,2020-09-30,83.0,AL,3,314,Floridan aquifer system,False,MURDER CREEK NEAR EVERGREEN AL,southeast,
8,18,02376500,1990-10-01,2020-09-30,30,0.9,True,0.000,1400.0,30,...,2020-09-30,79.3,AL,3,314,Coastal lowlands aquifer system,False,"PERDIDO RIVER AT BARRINEAU PARK, FL",southeast,
9,19,02377570,1990-10-01,2020-09-30,30,0.9,True,0.000,780.6,30,...,2020-09-30,33.0,AL,3,314,Coastal lowlands aquifer system,False,"STYX RIVER NEAR ELSANOR, AL.",southeast,


# Save mean, min, and max HMF metrics for areas

## HUC2 gages

In [53]:
huc2_codes = list(range(1, 19))
huc2_codes

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

### Mean

In [51]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'huc2_code']
print('Number of gages in CONUS (30-year record):', len(dfs_valid[f'30_90']))
print('Number of gages in CONUS (50-year record):', len(dfs_valid[f'50_90']))

df_mean_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df = dfs_valid[f'{date_range}_{percentile}']
        df_mean_metrics[f'{date_range}_{percentile}'] = df[metric_list].groupby('huc2_code').mean()
        df_mean_metrics[f'{date_range}_{percentile}']['type'] = 'mean'

Number of gages in CONUS (30-year record): 4242
Number of gages in CONUS (50-year record): 3314


In [41]:
# Save df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        file_name = f'huc2_hmf_metrics_mean_{date_range}_{percentile}.xlsx'
        df_mean_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Median

In [52]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'huc2_code']

df_median_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df = dfs_valid[f'{date_range}_{percentile}']
        df_median_metrics[f'{date_range}_{percentile}'] = df[metric_list].groupby('huc2_code').median()
        df_median_metrics[f'{date_range}_{percentile}']['type'] = 'median'

### Min

In [53]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'huc2_code']
#print('Number of gages in 25 most pumped aqs:', len(dfs_valid[f'{date_range}_{percentile}'].loc[dfs_valid[f'{date_range}_{percentile}']['within_aq'].isin(aq_names_25.values())]))

df_min_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df_min_metrics[f'{date_range}_{percentile}'] = dfs_valid[f'{date_range}_{percentile}'][metric_list].groupby('huc2_code').min()
        df_min_metrics[f'{date_range}_{percentile}']['type'] = 'min'

In [189]:
# # Save df to excel
# for date_range in date_ranges:
#     for percentile in percentiles:
#         file_name = f'huc2_hmf_metrics_min_{date_range}_{percentile}.xlsx'
#         df_min_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Max

In [54]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'huc2_code']
#print('Number of gages in 25 most pumped aqs:', len(dfs_valid[f'{date_range}_{percentile}'].loc[dfs_valid[f'{date_range}_{percentile}']['within_aq'].isin(aq_names_25.values())]))

df_max_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df_max_metrics[f'{date_range}_{percentile}'] = dfs_valid[f'{date_range}_{percentile}'][metric_list].groupby('huc2_code').max()
        df_max_metrics[f'{date_range}_{percentile}']['type'] = 'max'

In [191]:
# # Save df to excel
# for date_range in date_ranges:
#     for percentile in percentiles:
#         file_name = f'huc2_hmf_metrics_max_{date_range}_{percentile}.xlsx'
#         df_max_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Combine max, mean, median, and min tables and save as one excel file

In [69]:
# Save combined df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        # Combine the three dataframes
        df_combined = pd.concat([df_min_metrics[f'{date_range}_{percentile}'], 
                                 df_mean_metrics[f'{date_range}_{percentile}'],
                                 df_max_metrics[f'{date_range}_{percentile}']])

        # Sort the combined dataframe to ensure the order is min, mean, max for each aquifer
        df_combined = df_combined.sort_values(by=['huc2_code', 'type'])

        # Save the combined dataframe to a new Excel file
        file_name = f'huc2_hmf_metrics_{date_range}_{percentile}.xlsx'
        df_combined.to_excel('Tables/'+file_name)

In [55]:
# Save combined df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        # Combine the three dataframes
        df_combined = pd.concat([df_min_metrics[f'{date_range}_{percentile}'], 
                                 df_mean_metrics[f'{date_range}_{percentile}'],
                                 df_median_metrics[f'{date_range}_{percentile}'],
                                 df_max_metrics[f'{date_range}_{percentile}']])

        # Sort the combined dataframe to ensure the order is min, mean, max for each aquifer
        df_combined = df_combined.sort_values(by=['huc2_code', 'type'])

        # Save the combined dataframe to a new Excel file
        file_name = f'huc2_hmf_metrics_median_{date_range}_{percentile}.xlsx'
        df_combined.to_excel('Tables/'+file_name)

## Compare percentiles and date ranges

In [70]:
huc2_hmf_metrics_30_90 = pd.read_excel('Tables/huc2_hmf_metrics_30_90.xlsx')
huc2_hmf_metrics_30_90['type'] = huc2_hmf_metrics_30_90['type'].replace({'max': 1, 'mean': 2, 'min': 3})

huc2_hmf_metrics_30_95 = pd.read_excel('Tables/huc2_hmf_metrics_30_95.xlsx')
huc2_hmf_metrics_30_95['type'] = huc2_hmf_metrics_30_95['type'].replace({'max': 1, 'mean': 2, 'min': 3})

huc2_hmf_metrics_50_90 = pd.read_excel('Tables/huc2_hmf_metrics_50_90.xlsx')
huc2_hmf_metrics_50_90['type'] = huc2_hmf_metrics_50_90['type'].replace({'max': 1, 'mean': 2, 'min': 3})

huc2_hmf_metrics_50_95 = pd.read_excel('Tables/huc2_hmf_metrics_50_95.xlsx')
huc2_hmf_metrics_50_95['type'] = huc2_hmf_metrics_50_95['type'].replace({'max': 1, 'mean': 2, 'min': 3})

C:\Users\kondris\AppData\Local\Temp\ipykernel_120260\4053928047.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  huc2_hmf_metrics_30_90['type'] = huc2_hmf_metrics_30_90['type'].replace({'max': 1, 'mean': 2, 'min': 3})
C:\Users\kondris\AppData\Local\Temp\ipykernel_120260\4053928047.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  huc2_hmf_metrics_30_95['type'] = huc2_hmf_metrics_30_95['type'].replace({'max': 1, 'mean': 2, 'min': 3})
C:\Users\kondris\AppData\Local\Temp\ipykernel_120260\4053928047.py:8: FutureWarning: Downcasti

In [71]:
huc2_hmf_metrics_median_30_90 = pd.read_excel('Tables/huc2_hmf_metrics_median_30_90.xlsx')
huc2_hmf_metrics_median_30_90['type'] = huc2_hmf_metrics_median_30_90['type'].replace({'max': 1, 'mean': 2, 'median': 3, 'min': 4})

huc2_hmf_metrics_median_30_95 = pd.read_excel('Tables/huc2_hmf_metrics_median_30_95.xlsx')
huc2_hmf_metrics_median_30_95['type'] = huc2_hmf_metrics_median_30_95['type'].replace({'max': 1, 'mean': 2, 'median': 3, 'min': 4})

huc2_hmf_metrics_median_50_90 = pd.read_excel('Tables/huc2_hmf_metrics_median_50_90.xlsx')
huc2_hmf_metrics_median_50_90['type'] = huc2_hmf_metrics_median_50_90['type'].replace({'max': 1, 'mean': 2, 'median': 3, 'min': 4})

huc2_hmf_metrics_median_50_95 = pd.read_excel('Tables/huc2_hmf_metrics_median_50_95.xlsx')
huc2_hmf_metrics_median_50_95['type'] = huc2_hmf_metrics_median_50_95['type'].replace({'max': 1, 'mean': 2, 'median': 3, 'min': 4})

C:\Users\kondris\AppData\Local\Temp\ipykernel_120260\2219596975.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  huc2_hmf_metrics_median_30_90['type'] = huc2_hmf_metrics_median_30_90['type'].replace({'max': 1, 'mean': 2, 'median': 3, 'min': 4})
C:\Users\kondris\AppData\Local\Temp\ipykernel_120260\2219596975.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  huc2_hmf_metrics_median_30_95['type'] = huc2_hmf_metrics_median_30_95['type'].replace({'max': 1, 'mean': 2, 'median': 3, 'min': 4})
C:\Users\kondris\AppData\Local\Temp\ipyk

In [73]:
huc2_list = huc2_hmf_metrics_30_90['huc2_code'].tolist()
type_list = huc2_hmf_metrics_30_90['type'].tolist()

huc2_hmf_metrics_30_90_50 = ((huc2_hmf_metrics_30_90 - huc2_hmf_metrics_50_90) / huc2_hmf_metrics_30_90) * 100
huc2_hmf_metrics_30_90_50['huc2_code'] = huc2_list
huc2_hmf_metrics_30_90_50['type'] = type_list
file_name = f'huc2_hmf_metrics_30_90_50_compare.xlsx'
huc2_hmf_metrics_30_90_50.to_excel('Tables/'+file_name)

huc2_hmf_metrics_30_90_95 = ((huc2_hmf_metrics_30_90 - huc2_hmf_metrics_30_95) / huc2_hmf_metrics_30_90) * 100
huc2_hmf_metrics_30_90_95['huc2_code'] = huc2_hmf_metrics_30_90['huc2_code']
huc2_hmf_metrics_30_90_95['type'] = type_list
file_name = f'huc2_hmf_metrics_30_90_95_compare.xlsx'
huc2_hmf_metrics_30_90_95.to_excel('Tables/'+file_name)

In [74]:
huc2_list = huc2_hmf_metrics_median_30_90['huc2_code'].tolist()
type_list = huc2_hmf_metrics_median_30_90['type'].tolist()

huc2_hmf_metrics_median_30_90_50 = ((huc2_hmf_metrics_median_30_90 - huc2_hmf_metrics_median_50_90) / huc2_hmf_metrics_median_30_90) * 100
huc2_hmf_metrics_median_30_90_50['huc2_code'] = huc2_list
huc2_hmf_metrics_median_30_90_50['type'] = type_list
file_name = f'huc2_hmf_metrics_median_30_90_50_compare.xlsx'
huc2_hmf_metrics_median_30_90_50.to_excel('Tables/'+file_name)

huc2_hmf_metrics_median_30_90_95 = ((huc2_hmf_metrics_median_30_90 - huc2_hmf_metrics_median_30_95) / huc2_hmf_metrics_median_30_90) * 100
huc2_hmf_metrics_median_30_90_95['huc2_code'] = huc2_hmf_metrics_median_30_90['huc2_code']
huc2_hmf_metrics_median_30_90_95['type'] = type_list
file_name = f'huc2_hmf_metrics_median_30_90_95_compare.xlsx'
huc2_hmf_metrics_median_30_90_95.to_excel('Tables/'+file_name)

In [112]:
#huc2_hmf_metrics_30_90_50

## Regions

In [86]:
unique_region_list = dfs_valid[f'{date_range}_{percentile}']['region'].unique().tolist()
unique_region_list = sorted(unique_region_list)
unique_region_list

['midwest',
 'northeast',
 'pac_northwest',
 'southcentral',
 'southeast',
 'southwest',
 'upper_plains']

### Mean

In [77]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'huc2_code', 'region']
print('Number of gages in CONUS (30-year record):', len(dfs_valid[f'30_90']))
print('Number of gages in CONUS (50-year record):', len(dfs_valid[f'50_90']))

df_mean_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df = dfs_valid[f'{date_range}_{percentile}']
        df_mean_metrics[f'{date_range}_{percentile}'] = df[metric_list].groupby('region').mean()
        df_mean_metrics[f'{date_range}_{percentile}']['type'] = 'mean'

Number of gages in CONUS (30-year record): 4242
Number of gages in CONUS (50-year record): 3314


In [78]:
# Save df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        file_name = f'region_hmf_metrics_mean_{date_range}_{percentile}.xlsx'
        df_mean_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Median

In [79]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'huc2_code', 'region']

df_median_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df = dfs_valid[f'{date_range}_{percentile}']
        df_median_metrics[f'{date_range}_{percentile}'] = df[metric_list].groupby('region').median()
        df_median_metrics[f'{date_range}_{percentile}']['type'] = 'median'

### Min

In [80]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'huc2_code', 'region']
#print('Number of gages in 25 most pumped aqs:', len(dfs_valid[f'{date_range}_{percentile}'].loc[dfs_valid[f'{date_range}_{percentile}']['within_aq'].isin(aq_names_25.values())]))

df_min_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df_min_metrics[f'{date_range}_{percentile}'] = dfs_valid[f'{date_range}_{percentile}'][metric_list].groupby('region').min()
        df_min_metrics[f'{date_range}_{percentile}']['type'] = 'min'

In [81]:
# # Save df to excel
# for date_range in date_ranges:
#     for percentile in percentiles:
#         file_name = f'huc2_hmf_metrics_min_{date_range}_{percentile}.xlsx'
#         df_min_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Max

In [82]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'huc2_code', 'region']
#print('Number of gages in 25 most pumped aqs:', len(dfs_valid[f'{date_range}_{percentile}'].loc[dfs_valid[f'{date_range}_{percentile}']['within_aq'].isin(aq_names_25.values())]))

df_max_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df_max_metrics[f'{date_range}_{percentile}'] = dfs_valid[f'{date_range}_{percentile}'][metric_list].groupby('region').max()
        df_max_metrics[f'{date_range}_{percentile}']['type'] = 'max'

In [83]:
# # Save df to excel
# for date_range in date_ranges:
#     for percentile in percentiles:
#         file_name = f'huc2_hmf_metrics_max_{date_range}_{percentile}.xlsx'
#         df_max_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Combine max, mean, and min tables and save as one excel file

In [84]:
# Save combined df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        # Combine the three dataframes
        df_combined = pd.concat([df_min_metrics[f'{date_range}_{percentile}'], 
                                 df_mean_metrics[f'{date_range}_{percentile}'], 
                                 df_max_metrics[f'{date_range}_{percentile}']])

        # Sort the combined dataframe to ensure the order is min, mean, max for each aquifer
        df_combined = df_combined.sort_values(by=['region', 'type'])

        # Save the combined dataframe to a new Excel file
        file_name = f'region_hmf_metrics_{date_range}_{percentile}.xlsx'
        df_combined.to_excel('Tables/'+file_name)

In [85]:
# Save combined df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        # Combine the three dataframes
        df_combined = pd.concat([df_min_metrics[f'{date_range}_{percentile}'], 
                                 df_mean_metrics[f'{date_range}_{percentile}'],
                                 df_median_metrics[f'{date_range}_{percentile}'],
                                 df_max_metrics[f'{date_range}_{percentile}']])

        # Sort the combined dataframe to ensure the order is min, mean, max for each aquifer
        df_combined = df_combined.sort_values(by=['region', 'type'])

        # Save the combined dataframe to a new Excel file
        file_name = f'region_hmf_metrics_median_{date_range}_{percentile}.xlsx'
        df_combined.to_excel('Tables/'+file_name)

## Compare percentiles and date ranges

In [266]:
region_hmf_metrics_30_90 = pd.read_excel('Tables/region_hmf_metrics_30_90.xlsx')
region_hmf_metrics_30_90['type'] = region_hmf_metrics_30_90['type'].replace({'max': 1, 'mean': 2, 'min': 3})

region_hmf_metrics_30_95 = pd.read_excel('Tables/region_hmf_metrics_30_95.xlsx')
region_hmf_metrics_30_95['type'] = region_hmf_metrics_30_95['type'].replace({'max': 1, 'mean': 2, 'min': 3})

region_hmf_metrics_50_90 = pd.read_excel('Tables/region_hmf_metrics_50_90.xlsx')
region_hmf_metrics_50_90['type'] = region_hmf_metrics_50_90['type'].replace({'max': 1, 'mean': 2, 'min': 3})

region_hmf_metrics_50_95 = pd.read_excel('Tables/region_hmf_metrics_50_95.xlsx')
region_hmf_metrics_50_95['type'] = region_hmf_metrics_50_95['type'].replace({'max': 1, 'mean': 2, 'min': 3})

C:\Users\kondris\AppData\Local\Temp\ipykernel_224532\1773071955.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  region_hmf_metrics_30_90['type'] = region_hmf_metrics_30_90['type'].replace({'max': 1, 'mean': 2, 'min': 3})
C:\Users\kondris\AppData\Local\Temp\ipykernel_224532\1773071955.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  region_hmf_metrics_30_95['type'] = region_hmf_metrics_30_95['type'].replace({'max': 1, 'mean': 2, 'min': 3})
C:\Users\kondris\AppData\Local\Temp\ipykernel_224532\1773071955.py:8: FutureWarning: D

In [268]:
unique_region_list = region_hmf_metrics_30_90['region'].unique().tolist()
region_list = region_hmf_metrics_30_90['region'].tolist()
type_list = region_hmf_metrics_30_90['type'].tolist()

region_hmf_metrics_30_90 = region_hmf_metrics_30_90.drop('region', axis=1)
region_hmf_metrics_50_90 = region_hmf_metrics_50_90.drop('region', axis=1)
region_hmf_metrics_30_95 = region_hmf_metrics_30_95.drop('region', axis=1)

region_hmf_metrics_30_90_50 = ((region_hmf_metrics_30_90 - region_hmf_metrics_50_90) / region_hmf_metrics_30_90) * 100
region_hmf_metrics_30_90_50['region'] = region_list
region_hmf_metrics_30_90_50['type'] = type_list
file_name = f'region_hmf_metrics_30_90_50_compare.xlsx'
region_hmf_metrics_30_90_50.to_excel('Tables/'+file_name)

region_hmf_metrics_30_90_95 = ((region_hmf_metrics_30_90 - region_hmf_metrics_30_95) / region_hmf_metrics_30_90) * 100
region_hmf_metrics_30_90_95['region'] = region_list
region_hmf_metrics_30_90_95['type'] = type_list
file_name = f'region_hmf_metrics_30_90_95_compare.xlsx'
region_hmf_metrics_30_90_95.to_excel('Tables/'+file_name)

## Aquifer

In [83]:
## Aquifer gages
#dfs_valid['30_90']['within_aq'].unique()

dfs_aq = {}
date_ranges = ['30', '50']
percentiles = ['90', '95']
for date_range in date_ranges:
    for percentile in percentiles: 
        df = dfs_valid[f'{date_range}_{percentile}']
        dfs_aq[f'{date_range}_{percentile}'] = df.loc[df['within_aq'].isin(aq_names_25_list)]
print(len(dfs_aq['30_90']))
print(len(dfs_aq['50_90']))

dfs_aq_10 = {}
date_ranges = ['30', '50']
percentiles = ['90', '95']
for date_range in date_ranges:
    for percentile in percentiles: 
        df = dfs_valid[f'{date_range}_{percentile}']
        dfs_aq_10[f'{date_range}_{percentile}'] = df.loc[df['within_aq'].isin(aq_names_10_list)]
print(len(dfs_aq_10['30_90']))
print(len(dfs_aq_10['50_90']))

dfs_aq_selected = {}
date_ranges = ['30', '50']
percentiles = ['90', '95']
for date_range in date_ranges:
    for percentile in percentiles: 
        df = dfs_valid[f'{date_range}_{percentile}']
        dfs_aq_selected[f'{date_range}_{percentile}'] = df.loc[df['within_aq'].isin(aq_names_selected_list)]
print(len(dfs_aq_selected['30_90']))
print(len(dfs_aq_selected['50_90']))

1690
1299
624
490
166
128


In [150]:
counts = dfs_aq['30_90']['within_aq'].value_counts()
counts_df = counts.reset_index()
counts_df.columns = ['within_aq', 'count']
counts_df.to_excel('Tables/aq_counts_30_90.xlsx')

counts = dfs_aq['50_90']['within_aq'].value_counts()
counts_df = counts.reset_index()
counts_df.columns = ['within_aq', 'count']
counts_df.to_excel('Tables/aq_counts_50_90.xlsx')

In [31]:
# Save df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        file_name = f'aq_all_gages_{date_range}_{percentile}.xlsx'
        dfs_aq[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)
        
for date_range in date_ranges:
    for percentile in percentiles:
        file_name = f'10_aq_all_gages_{date_range}_{percentile}.xlsx'
        dfs_aq_10[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Mean

In [112]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'within_aq']
print('Number of gages in 25 most pumped aqs:', len(dfs_valid[f'{date_range}_{percentile}'].loc[dfs_valid[f'{date_range}_{percentile}']['within_aq'].isin(aq_names_25.values())]))

df_mean_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df_mean_metrics[f'{date_range}_{percentile}'] = dfs_valid[f'{date_range}_{percentile}'][metric_list].groupby('within_aq').mean()
        df_mean_metrics[f'{date_range}_{percentile}']['type'] = 'mean'

Number of gages in 25 most pumped aqs: 1299


In [113]:
# # Save df to excel
# for date_range in date_ranges:
#     for percentile in percentiles:
#         file_name = f'aq_hmf_metrics_all_mean_{date_range}_{percentile}.xlsx'
#         df_mean_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Min

In [114]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'within_aq']
print('Number of gages in 25 most pumped aqs:', len(dfs_valid[f'{date_range}_{percentile}'].loc[dfs_valid[f'{date_range}_{percentile}']['within_aq'].isin(aq_names_25.values())]))

df_min_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df_min_metrics[f'{date_range}_{percentile}'] = dfs_valid[f'{date_range}_{percentile}'][metric_list].groupby('within_aq').min()
        df_min_metrics[f'{date_range}_{percentile}']['type'] = 'min'

Number of gages in 25 most pumped aqs: 1299


In [115]:
# # Save df to excel
# for date_range in date_ranges:
#     for percentile in percentiles:
#         file_name = f'aq_hmf_metrics_all_min_{date_range}_{percentile}.xlsx'
#         df_min_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Max

In [116]:
# All gages in each aquifer
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'within_aq']
print('Number of gages in 25 most pumped aqs:', len(dfs_valid[f'{date_range}_{percentile}'].loc[dfs_valid[f'{date_range}_{percentile}']['within_aq'].isin(aq_names_25.values())]))

df_max_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df_max_metrics[f'{date_range}_{percentile}'] = dfs_valid[f'{date_range}_{percentile}'][metric_list].groupby('within_aq').max()
        df_max_metrics[f'{date_range}_{percentile}']['type'] = 'max'

Number of gages in 25 most pumped aqs: 1299


In [117]:
# # Save df to excel
# for date_range in date_ranges:
#     for percentile in percentiles:
#         file_name = f'aq_hmf_metrics_all_max_{date_range}_{percentile}.xlsx'
#         df_max_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Combine max, mean, and min tables and save as one excel file

In [118]:
# Save combined df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        # Combine the three dataframes
        df_combined = pd.concat([df_min_metrics[f'{date_range}_{percentile}'], 
                                 df_mean_metrics[f'{date_range}_{percentile}'], 
                                 df_max_metrics[f'{date_range}_{percentile}']])

        # Sort the combined dataframe to ensure the order is min, mean, max for each aquifer
        df_combined = df_combined.sort_values(by=['within_aq', 'type'])

        # Save the combined dataframe to a new Excel file
        file_name = f'aq_hmf_metrics_all_{date_range}_{percentile}.xlsx'
        df_combined.to_excel('Tables/'+file_name)

In [113]:
aq_hmf_metrics_all_30_90 = pd.read_excel('Tables/aq_hmf_metrics_all_30_90.xlsx')
aq_hmf_metrics_all_30_90['type'] = aq_hmf_metrics_all_30_90['type'].replace({'max': 1, 'mean': 2, 'min': 3})

aq_hmf_metrics_all_30_95 = pd.read_excel('Tables/aq_hmf_metrics_all_30_95.xlsx')
aq_hmf_metrics_all_30_95['type'] = aq_hmf_metrics_all_30_95['type'].replace({'max': 1, 'mean': 2, 'min': 3})

aq_hmf_metrics_all_50_90 = pd.read_excel('Tables/aq_hmf_metrics_all_50_90.xlsx')
aq_hmf_metrics_all_50_90['type'] = aq_hmf_metrics_all_50_90['type'].replace({'max': 1, 'mean': 2, 'min': 3})

aq_hmf_metrics_all_50_95 = pd.read_excel('Tables/aq_hmf_metrics_all_50_95.xlsx')
aq_hmf_metrics_all_50_95['type'] = aq_hmf_metrics_all_50_95['type'].replace({'max': 1, 'mean': 2, 'min': 3})

C:\Users\kondris\AppData\Local\Temp\ipykernel_224532\1868654585.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aq_hmf_metrics_all_30_90['type'] = aq_hmf_metrics_all_30_90['type'].replace({'max': 1, 'mean': 2, 'min': 3})
C:\Users\kondris\AppData\Local\Temp\ipykernel_224532\1868654585.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aq_hmf_metrics_all_30_95['type'] = aq_hmf_metrics_all_30_95['type'].replace({'max': 1, 'mean': 2, 'min': 3})
C:\Users\kondris\AppData\Local\Temp\ipykernel_224532\1868654585.py:8: FutureWarning: D

In [116]:
aq_list = aq_hmf_metrics_all_30_90['within_aq'].tolist()
type_list = aq_hmf_metrics_all_30_90['type'].tolist()

In [120]:
aq_hmf_metrics_all_30_90 = aq_hmf_metrics_all_30_90.drop('within_aq', axis=1)
aq_hmf_metrics_all_50_90 = aq_hmf_metrics_all_50_90.drop('within_aq', axis=1)
aq_hmf_metrics_all_30_95 = aq_hmf_metrics_all_30_95.drop('within_aq', axis=1)

aq_hmf_metrics_all_30_90_50 = ((aq_hmf_metrics_all_30_90 - aq_hmf_metrics_all_50_90) / aq_hmf_metrics_all_30_90) * 100
aq_hmf_metrics_all_30_90_50['within_aq'] = aq_list
aq_hmf_metrics_all_30_90_50['type'] = type_list
file_name = f'aq_hmf_metrics_all_30_90_50.xlsx'
aq_hmf_metrics_all_30_90_50.to_excel('Tables/'+file_name)

aq_hmf_metrics_all_30_90_95 = ((aq_hmf_metrics_all_30_90 - aq_hmf_metrics_all_30_95) / aq_hmf_metrics_all_30_90) * 100
aq_hmf_metrics_all_30_90_95['within_aq'] = aq_list
aq_hmf_metrics_all_30_90_95['type'] = type_list
file_name = f'aq_hmf_metrics_all_30_90_95.xlsx'
aq_hmf_metrics_all_30_90_95.to_excel('Tables/'+file_name)

## Outlet gages

In [70]:
# NEW huc4 outlet gages by aquifer
hp_outlet_gages = ['06447000', '06465500', '06674500', '06764880', '06801000', '06793000', '06800500', '06853500', '06873000', '06881000', 
                   '07134180', '07144300', '07157500', '07227500', '07235000', '07297910']
mr_outlet_gages = ['07022000', '07077000', '07288280', '07364200', '07369000', '07074500']
cv_outlet_gages = ['11447650', '11200800', '11303500']
br_outlet_gages = ['09415000', '09521100', '09466500', '09469500', '09502000', '09519800', '09537200', '10126000', '10141000', '10219000', 
                   '10327500', '10351700', '10249280', '13105000', '11062000', '10260500', '10254730']
fl_outlet_gages = ['02240000', '02313000', '02320500', '02326900', '02358000', '02365500']
sr_outlet_gages = ['13087995', '13269000']
cl_outlet_gages = ['02375500', '02471001', '02479000', '02489500', '07373000', '07290000', '07378500', '08013500', '07375000', '08030500', 
                   '08033500', '08066500', '08068000', '08116650', '08162000', '08176500', '08211000']
cc_outlet_gages = ['11477000', '11451000', '11458000', '11152500', '11078000']
pn_outlet_gages = ['10039500', '10172952', '10346000', '12398600', '12447200', '13087995', '13269000', '13317000', '14038530', '14233500', 
                   '14187200', '14321000', '12045500', '10396000', '11509500', '11377100']
na_outlet_gages = ['01402000', '01463500', '01578310', '01594440', '01646500', '02037500', '02080500', '02083500', '02105500', '02129000']
cp_outlet_gages = ['12422500', '12472800', '12510500', '13334300', '14105700']
rg_outlet_gages = ['08317400', '08361000']
me_outlet_gages = ['02428400', '02469761', '02477000', '02482550', '03611260', '03610200', '07029500', '07047942', '07268000', '07362000', 
                   '07289350', '07337000']
co_outlet_gages = ['04024000', '04045500', '04082400', '05331000', '05330000', '05340500', '05378500', '05369500', '05388250', '05404000', 
                   '05437500', '05514500', '05552500', '05555300']
sc_outlet_gages = ['02130561', '02169500', '02197000', '02223500', '02343801', '02373000', '02425000', '02467000', '03399800', '03610200']
bi_outlet_gages = ['02289060']
et_outlet_gages = ['07316000', '07335300', '08044500', '08091000', '08102500', '08123850', '08158000', '08167800', '08190000', '08446500']
rm_outlet_gages = ['06036650', '06054500', '06192500', '12395500', '13142500', '13247500', '13302500']
pb_outlet_gages = ['01388500', '01463500', '01578310', '01594000', '01646500', '02037500', '02066000', '02082585', '02102500', '02129000', 
                   '02169500', '02192000', '02213000', '02341460', '02414715', '03168000', '03453500', '03550000']
sa_outlet_gages = ['02105500', '02131000', '02171500', '02198500', '02226000', '02244040', '02292900', '02296750', '02315500', '02330000', 
                   '02359170', '02366500']
vr_outlet_gages = ['01446500', '01570500', '01613000', '02019500', '02056000', '02397000', '02455000', '03176500', '03540500', '03568933']
sd_outlet_gages = ['04056500', '04087000', '04126740', '04127997', '04166500', '04193500', '04127917', '03227500', '03270500', '03237500', 
                   '03283500', '03335500', '03434500', '03294500', '03604000', '05412500', '05420500', '05446500', '05587450', '05527500', 
                   '05570910']
pnb_outlet_gages = ['06037500', '06188000', '10352500', '12414500', '12488500', '13062500', '13269000', '13341050', '14046500', '14246900', 
                    '14211720', '14301000', '12205000', '10396000', '11509500', '11377100']
wl_outlet_gages = ['14142500', '14211720']
lc_outlet_gages = ['05270700', '05311000', '05449500', '05476750', '05082500', '06090300', '06208500', '06287000', '06436000', '06479010', 
                   '06630000', '06801000', '06800500', '06486000', '06809500', '06856600', '06876900', '06884025', '07140850']

hp_n_outlet_gages = ['06447000', '06465500', '06674500', '06764880', '06801000', '06793000', '06800500', '06853500', '06881000']
hp_c_s_outlet_gages = ['06873000', '07134180', '07144300', '07157500', '07227500', '07235000', '07297910']
cv_sac_outlet_gages = ['11447650']
cv_sj_outlet_gages = ['11303500']
cl_tx_outlet_gages = ['08211000', '08188500', '08176500', '08164000', '08162000', '08116650', '08066500', '08033500', '08030500', '08068000']
cl_cl_outlet_gages = ['02375500', '02471001', '02479000', '02489500', '07373000', '07290000', '07378500', '07375000', '08013500']
br_az_outlet_gages = ['09415000', '09521100', '09466500', '09469500', '09502000', '09519800', '09537200'] #['09520500', '09521100', '09519800', '09429100']

outlet_gages_dict_10 = {
    'br': br_outlet_gages, 
    'cc': cc_outlet_gages,
    'cv': cv_outlet_gages,
    'cl': cl_outlet_gages,
    'fl': fl_outlet_gages,
    'hp': hp_outlet_gages,
    'mr': mr_outlet_gages,
    'na': na_outlet_gages,
    'pn': pn_outlet_gages,
    'sr': sr_outlet_gages
}

outlet_gages_dict_25 = {
    'br': br_outlet_gages, 
    'cc': cc_outlet_gages,
    'cv': cv_outlet_gages,
    'cl': cl_outlet_gages,
    'fl': fl_outlet_gages,
    'hp': hp_outlet_gages,
    'mr': mr_outlet_gages,
    'na': na_outlet_gages,
    'pn': pn_outlet_gages,
    'sr': sr_outlet_gages,
    'cp': cp_outlet_gages,
    'rg': rg_outlet_gages,
    'me': me_outlet_gages,
    'co': co_outlet_gages,
    'sc': sc_outlet_gages,
    'bi': bi_outlet_gages,
    'et': et_outlet_gages,
    'rm': rm_outlet_gages,
    'pb': pb_outlet_gages,
    'sa': sa_outlet_gages,
    'vr': vr_outlet_gages,
    'sd': sd_outlet_gages,
    'pnb': pnb_outlet_gages,
    'wl': wl_outlet_gages,
    'lc': lc_outlet_gages,
    'hp_n': hp_n_outlet_gages,
    'hp_c_s': hp_c_s_outlet_gages,
    'cv_sac': cv_sac_outlet_gages,
    'cv_sj': cv_sj_outlet_gages,
    'cl_tx': cl_tx_outlet_gages,
    'cl_cl': cl_cl_outlet_gages,
    'br_az': br_az_outlet_gages
}

aq_names_10 = {'hp': 'High Plains aquifer',
                'mr': 'Mississippi River Valley alluvial aquifer',
                'cv': 'Central Valley aquifer system',
                'br': 'Basin and Range basin-fill aquifers',
                'fl': 'Floridan aquifer system',
                'sr': 'Snake River Plain basaltic-rock aquifers',
                'cl': 'Coastal lowlands aquifer system',
                'cc': 'California Coastal Basin aquifers', 
                'pn': 'Pacific Northwest basin-fill aquifers',
                'na': 'Northern Atlantic Coastal Plain aquifer system'}

aq_names_25 = {'hp': 'High Plains aquifer',
                'mr': 'Mississippi River Valley alluvial aquifer',
                'cv': 'Central Valley aquifer system',
                'br': 'Basin and Range basin-fill aquifers',
                'fl': 'Floridan aquifer system',
                'sr': 'Snake River Plain basaltic-rock aquifers',
                'cl': 'Coastal lowlands aquifer system',
                'cc': 'California Coastal Basin aquifers', 
                'pn': 'Pacific Northwest basin-fill aquifers',
                'na': 'Northern Atlantic Coastal Plain aquifer system',
                'cp': 'Columbia Plateau basaltic-rock aquifers',
                'rg': 'Rio Grande aquifer system',
                'me': 'Mississippi embayment aquifer system',
                'co': 'Cambrian-Ordovician aquifer system',
                'sc': 'Southeastern Coastal Plain aquifer system',
                'bi': 'Biscayne aquifer',
                'et': 'Edwards-Trinity aquifer system',
                'rm': 'Northern Rocky Mountains Intermontane Basins aquifer system',
                'pb': 'Piedmont and Blue Ridge crystalline-rock aquifers',
                'sa': 'Surficial aquifer system',
                'vr': 'Valley and Ridge aquifers',
                'sd': 'Silurian-Devonian aquifers',
                'pnb': 'Pacific Northwest basaltic-rock aquifers',
                'wl': 'Willamette Lowland basin-fill aquifers',
                'lc': 'Lower Cretaceous aquifers',
                'hp_n': 'Northern High Plaisn',
                'hp_c_s': 'Central South High Plains',
                'cv_sac': 'Sacramento River Basin',
                'cv_sj': 'San Joaquin River Basin',
                'cl_tx': 'Texas Gulf Coast',
                'cl_cl': 'Coastal Lowlands',
               'br_az': 'Arizona Alluvial'
              }

aq_codes_10 = ['hp', 'mr', 'cv', 'br', 'fl', 'sr', 'cl', 'cc', 'pn', 'na']

aq_codes_25 = ['hp', 'mr', 'cv', 'br', 'fl', 'sr', 'cl', 'cc', 'pn', 'na',
              'cp', 'rg', 'me', 'co', 'sc', 'bi', 'et', 'rm', 'pb', 'sa',
              'vr', 'sd', 'pnb', 'wl', 'lc', 'hp_n', 'hp_c_s', 'cv_sac', 'cv_sj',
              'cl_tx', 'cl_cl', 'br_az']

In [71]:
df_outlet_gages = {}
for date_range in date_ranges:
    for percentile in percentiles: 
        df_outlet_gages_aq = {}
        df_temp = dfs_valid[f'{date_range}_{percentile}']

        for key, value in outlet_gages_dict_25.items():
            df_outlet_gages_aq[key] = df_temp[df_temp['site_no'].isin(value)].copy()
            #print(df_outlet_gages_aq)
            # Add the key as a column to each DataFrame
            df_outlet_gages_aq[key].loc[:, 'aq'] = aq_names_25[key]

            #print(df_outlet_gages_aq)
       
        df_outlet_gages[f'{date_range}_{percentile}'] = df_outlet_gages_aq

In [72]:
df_outlet_gages['30_90']['br_az']['annual_hmf']

81     0.057205
94     0.237889
96     0.192242
125    0.245282
150    1.063235
166    0.000821
174    0.245846
Name: annual_hmf, dtype: float64

In [61]:
data = df_outlet_gages['30_90']
sub_key_counts = {}
for name, sub_key in data.items():
    count = sub_key['site_no'].count()  # Count non-NA entries in 'sub_key'
    sub_key_counts[aq_names_25[name]] = count
counts_df = pd.DataFrame.from_dict(sub_key_counts, orient='index', columns=['sub_key_count'])
counts_df.to_excel('Tables/aq_counts_outlet_gages_30_90.xlsx', index=True)

data = df_outlet_gages['50_90']
sub_key_counts = {}
for name, sub_key in data.items():
    count = sub_key['site_no'].count()  # Count non-NA entries in 'sub_key'
    sub_key_counts[aq_names_25[name]] = count
counts_df = pd.DataFrame.from_dict(sub_key_counts, orient='index', columns=['sub_key_count'])
counts_df.to_excel('Tables/aq_counts_outlet_gages_50_90.xlsx', index=True)

### Mean

In [62]:
# Mean HMF metrics: OUTLET GAGES (by aquifer)
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'aq']

df_mean_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df = pd.concat(df_outlet_gages[f'{date_range}_{percentile}'].values(), ignore_index=True)
        df_mean_metrics[f'{date_range}_{percentile}'] = df[metric_list].groupby('aq').mean()
        df_mean_metrics[f'{date_range}_{percentile}']['type'] = 'mean'

# OLD CODE
# metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
#                'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
#               'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
#                'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf']
#
#         data = {metric: [] for metric in metric_list}
        
#         # Loop through each aq code and metric to calculate the average and store in the dictionary
#         for aq in aq_codes_25:
#             for metric in metric_list:
#                 avg = df_outlet_gages[f'{date_range}_{percentile}'][aq][metric].mean()
#                 data[metric].append(avg)
        
#         # Convert the dictionary to a DataFrame
#         df_mean_metrics[f'{date_range}_{percentile}'] = pd.DataFrame(data, index=list(aq_names_25.values())) # index can also =aq_codes_25
#         df_mean_metrics[f'{date_range}_{percentile}']['type'] = 'mean'

In [63]:
# Save df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        file_name = f'aq_hmf_metrics_outlet_mean_{date_range}_{percentile}.xlsx'
        df_mean_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Min

In [33]:
# Min HMF metrics: OUTLET GAGES (by aquifer)
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'aq']

df_min_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df = pd.concat(df_outlet_gages[f'{date_range}_{percentile}'].values(), ignore_index=True)
        df_min_metrics[f'{date_range}_{percentile}'] = df[metric_list].groupby('aq').min()
        df_min_metrics[f'{date_range}_{percentile}']['type'] = 'min'

In [34]:
# Save df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        file_name = f'aq_hmf_metrics_outlet_min_{date_range}_{percentile}.xlsx'
        df_min_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Max

In [35]:
# Max HMF metrics: OUTLET GAGES (by aquifer)
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf', 'aq']

df_max_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df = pd.concat(df_outlet_gages[f'{date_range}_{percentile}'].values(), ignore_index=True)
        df_max_metrics[f'{date_range}_{percentile}'] = df[metric_list].groupby('aq').max()
        df_max_metrics[f'{date_range}_{percentile}']['type'] = 'max'

In [36]:
# Save df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        file_name = f'aq_hmf_metrics_outlet_max_{date_range}_{percentile}.xlsx'
        df_max_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

### Combine max, mean, and min tables and save as one excel file

In [37]:
# Save combined df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        # Combine the three dataframes
        df_combined = pd.concat([df_min_metrics[f'{date_range}_{percentile}'], 
                                 df_mean_metrics[f'{date_range}_{percentile}'], 
                                 df_max_metrics[f'{date_range}_{percentile}']])

        # Sort the combined dataframe to ensure the order is min, mean, max for each aquifer
        df_combined = df_combined.sort_values(by=['aq', 'type'])

        # Save the combined dataframe to a new Excel file
        file_name = f'aq_hmf_metrics_outlet_{date_range}_{percentile}.xlsx'
        df_combined.to_excel('Tables/'+file_name)

### Sum of annual HMF volume

In [73]:
# Sum of annual HMF: OUTLET gages (by aquifer)
metric = 'annual_hmf'
df_sum_annual_hmf = {}
for date_range in date_ranges:
    for percentile in percentiles:

        # Loop through each aq code to calculate the sum of the annual_hmf
        sum_list = []
        for aq in aq_codes_25:
            sums = df_outlet_gages[f'{date_range}_{percentile}'][aq][metric].sum()
            sum_list.append(sums)
            
        df_sum_annual_hmf[f'{date_range}_{percentile}'] = pd.DataFrame(sum_list, list(aq_names_25.values()))

In [74]:
df_outlet_gages[f'{30}_{90}']['cv']['annual_hmf']

380    0.011858
405    1.881519
432    2.632442
Name: annual_hmf, dtype: float64

In [75]:
dfs_dict = {
    "30_90": df_sum_annual_hmf['30_90'].squeeze(), #[df_sum_annual_hmf['30_90'].columns[1]],
    "30_95": df_sum_annual_hmf['30_95'].squeeze(),
    "50_90": df_sum_annual_hmf['50_90'].squeeze(),
    "50_95": df_sum_annual_hmf['50_95'].squeeze()
}

# Convert dictionary of Series into a single dataframe
pd.DataFrame(dfs_dict)

,30_90,30_95,50_90,50_95
High Plains aquifer,2.686754,1.922445,1.678764,1.220041
Mississippi River Valley alluvial aquifer,18.309537,10.932719,15.091744,10.176703
Central Valley aquifer system,4.525818,2.035843,4.682141,2.220351
Basin and Range basin-fill aquifers,2.940535,2.490168,4.632776,4.307282
Floridan aquifer system,4.484408,2.897048,3.908935,2.413973
Snake River Plain basaltic-rock aquifers,2.944964,1.777613,1.873736,0.976935
Coastal lowlands aquifer system,13.230223,8.183620,12.725935,7.938075
California Coastal Basin aquifers,2.963331,1.930543,3.268028,2.105450
Pacific Northwest basin-fill aquifers,11.986042,7.284171,11.304893,6.802659
Northern Atlantic Coastal Plain aquifer system,12.452212,7.616967,12.557969,7.595209


In [191]:
# Save df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        file_name = f'aq_annual_hmf_sum_outlet_sr6_{date_range}_{percentile}.xlsx'
        df_sum_annual_hmf[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

## MAR gages

In [18]:
# MAR gages
mar_gages = ['11152500', '11274000', '13087995', '13142500', '13147900', '14018500', '06711565', '06764880', '09486520', '09486520', '07077555',
             '08175800']
mar_gages_string = list(map(int, mar_gages))

df_mar = {}
date_ranges = ['30', '50']
percentiles = ['90', '95']
for date_range in date_ranges:
    for percentile in percentiles: 
        # Assuming df is your DataFrame and 'column_name' is the name of the column with numbers
        df = dfs_valid[f'{date_range}_{percentile}']
        df_mar[f'{date_range}_{percentile}'] = df[df['site_no'].isin(mar_gages)]


In [19]:
# Mean HMF metrics: MAR GAGES
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing',
              'jan_hmf', 'feb_hmf', 'mar_hmf', 'apr_hmf', 'may_hmf', 'jun_hmf', 'jul_hmf',
               'aug_hmf', 'sep_hmf', 'oct_hmf', 'nov_hmf', 'dec_hmf']

df_boxplot_metrics = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df = df_mar[f'{date_range}_{percentile}']
        df_boxplot_metrics[f'{date_range}_{percentile}'] = df[metric_list].describe()

# Save df to excel
for date_range in date_ranges:
    for percentile in percentiles:
        file_name = f'mar_metrics_{date_range}_{percentile}.xlsx'
        df_boxplot_metrics[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

# Count valid gages

In [45]:
df_gages_count = pd.DataFrame()
all_30 = len(dfs_metrics['30_90'])
all_50 = len(dfs_metrics['50_90'])
valid_30 = len(dfs_valid['30_90'][dfs_valid['30_90']['valid'] == True])
valid_50 = len(dfs_valid['50_90'][dfs_valid['50_90']['valid'] == True])
df_gages_count['data_range'] = ['30', '50']
df_gages_count['all_gages'] = [all_30, all_50]
df_gages_count['valid_gages'] = [valid_30, valid_50]
df_gages_count

NameError: name 'dfs_metrics' is not defined

In [48]:
date_range = '30'
percentile = '90'
df_aq_count_30_90 = pd.DataFrame()

num_gages_aq = []
num_gages_aq_outlet = []
for aq in aq_codes_25:
    num_gages_aq.append(len(dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['within_aq'] == aq_names_10[aq]]))
    num_gages_aq_outlet.append(len(df_outlet_gages[f'{date_range}_{percentile}'][aq]))

df_aq_count_30_90['aq'] = aq_names_10.keys()    
df_aq_count_30_90['valid_all'] = num_gages_aq
df_aq_count_30_90['valid_outlet'] = num_gages_aq_outlet

date_range = '50'
percentile = '90'
df_aq_count_50_90 = pd.DataFrame()

num_gages_aq = []
num_gages_aq_outlet = []
for aq in aq_codes_25:
    num_gages_aq.append(len(dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['within_aq'] == aq_names_10[aq]]))
    num_gages_aq_outlet.append(len(df_outlet_gages[f'{date_range}_{percentile}'][aq]))

df_aq_count_50_90['aq'] = aq_names_10.keys()    
df_aq_count_50_90['valid_all'] = num_gages_aq
df_aq_count_50_90['valid_outlet'] = num_gages_aq_outlet

NameError: name 'df_outlet_gages' is not defined

In [49]:
date_range = '30'
percentile = '90'
df_huc2_count_30_90 = pd.DataFrame()

num_gages_huc2 = []
for huc2 in huc2_codes:
    num_gages_huc2.append(len(dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['huc2_code'] == huc2]))

df_huc2_count_30_90['huc2'] = huc2_codes   
df_huc2_count_30_90['valid_all'] = num_gages_huc2

date_range = '50'
percentile = '90'
df_huc2_count_50_90 = pd.DataFrame()

num_gages_huc2 = []
for huc2 in huc2_codes:
    num_gages_huc2.append(len(dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['huc2_code'] == huc2]))

df_huc2_count_50_90['huc2'] = huc2_codes      
df_huc2_count_50_90['valid_all'] = num_gages_huc2

NameError: name 'huc2_codes' is not defined

In [47]:
print(df_huc2_count_30_90)
print(df_huc2_count_50_90)
print(df_huc2_count_30_90.sum())
print(df_huc2_count_50_90.sum())

    huc2  valid_all
0      1        164
1      2        440
2      3        549
3      4        234
4      5        324
5      6         47
6      7        319
7      8         49
8      9         60
9     10        399
10    11        259
11    12        211
12    13         71
13    14        159
14    15        147
15    16        127
16    17        446
17    18        237
    huc2  valid_all
0      1        144
1      2        360
2      3        361
3      4        184
4      5        289
5      6         38
6      7        255
7      8         46
8      9         53
9     10        312
10    11        197
11    12        177
12    13         61
13    14        119
14    15         99
15    16         90
16    17        328
17    18        201
huc2          171
valid_all    4242
dtype: int64
huc2          171
valid_all    3314
dtype: int64


## Import annual subdfs

In [123]:
subdfs_metrics_30_90 = pd.read_parquet(f'Prelim_Data/annual_metrics_subdf_30_90.parquet', engine='pyarrow') # only includes valid gages

In [134]:
annual_df_30_90 = subdfs_metrics_30_90[['site_no', 'water_year', 'annual_hmf']]
#annual_df_30_90 = annual_df_30_90.pivot(index='site_no', columns='water_year', values='annual_hmf')
annual_df_30_90

,site_no,water_year,annual_hmf
0,02342500,1990,0.092031
1,02342500,1991,0.067863
2,02342500,1992,0.125069
3,02342500,1993,0.164619
4,02342500,1994,0.048014
...,...,...,...
25,13018300,2015,0.000051
26,13018300,2016,0.007294
27,13018300,2017,0.003553
28,13018300,2018,0.001694


In [93]:
nan_rows = pivot_df[pivot_df.isna().any(axis=1)]
len(nan_rows)

180

In [69]:
# Only 
len(list(subdfs_metrics_30_90['site_no'].unique()))
total_hmf_30_90 = subdfs_metrics_30_90['annual_hmf'].sum()
print(f'Total HMF (30-year, 90th percentile): {total_hmf_30_90} km3')

Total HMF (30-year, 90th percentile): 23658.686901018093 km3


## Sort data

In [60]:
data_range = '30'
percentile = '90'
top_50_annual_hmf = dfs_valid[f'{date_range}_{percentile}'].sort_values(by=['annual_hmf'], ascending=False)[0:50]
#top_50_annual_hmf

## Describe data

In [28]:
date_range = '30'
percentile = '90'
metric = 'annual_hmf'

# CONUS
df = dfs_valid[f'{date_range}_{percentile}']

# Top 25
#df = dfs_aq[f'{date_range}_{percentile}']

# Top 10
#df = dfs_aq[f'{date_range}_{percentile}']
#df = df.loc[df['within_aq'].isin(aq_names_10)]

n = 15
high_value = 100
top_value = df.sort_values(by=[metric], ascending=False)[0:n].reset_index()
high_value = df[df[metric] > high_value].reset_index()

high_value = 0.15
low_value = 0.015
mid_annual_value = df[(df[metric] <= high_value) & (df[metric] >= low_value)].reset_index()

#low_annual_hmf = 

print(len(mid_annual_value))
print(len(df))
print(len(mid_annual_value) / len(df) * 100)
#high_annual_hmf

df[metric].describe()

2057
4242
48.49127769919849


count    4242.000000
mean        0.209971
std         0.682611
min         0.000003
25%         0.015643
50%         0.053223
75%         0.162787
max        14.069366
Name: annual_hmf, dtype: float64

In [29]:
date_range = '30'
percentile = '90'
metric = 'annual_hmf'
n = 164
top_annual_hmf = dfs_valid[f'{date_range}_{percentile}'].sort_values(by=[metric], ascending=False)[0:n].reset_index()
high_annual_hmf = dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['annual_hmf'] > 3.0].reset_index()
print(len(high_annual_hmf))
print(len(dfs_valid[f'{date_range}_{percentile}']))
print(len(high_annual_hmf) / len(dfs_valid[f'{date_range}_{percentile}']) * 100)
#high_annual_hmf

# Greater than 4 km3 - Missouri R, Ohio R, Susequehana R, Columbia R

32
4242
0.7543611504007544


# Boxplot tables for all gages and gages by aquifer and huc2

In [176]:
metric_list = ['annual_hmf', 'annual_duration', 'event_duration',
               'event_hmf', 'inter_annual%', 'intra_annual', 'timing']

# All valid gages
df_gages_boxplot = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df_metrics = {}
        for metric in metric_list:
            temp_df = dfs_valid[f'{date_range}_{percentile}'][metric]
            temp_df = temp_df.describe()
            df_metrics[metric] = temp_df
        df_gages_boxplot[f'{date_range}_{percentile}'] = df_metrics

# All valid aquifer gages (25 grouped)
df_aq25_grouped_boxplot = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df_metrics = {}
        for metric in metric_list:
            temp_df = dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['within_aq'].isin(aq_names_25_list)]
            temp_df = temp_df[metric].describe()
            df_metrics[metric] = temp_df
        df_aq25_grouped_boxplot[f'{date_range}_{percentile}'] = df_metrics

# All valid aquifer gages (10 grouped)
df_aq10_grouped_boxplot = {}
for date_range in date_ranges:
    for percentile in percentiles:
        df_metrics = {}
        for metric in metric_list:
            temp_df = dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['within_aq'].isin(aq_names_10_list)]
            temp_df = temp_df[metric].describe()
            df_metrics[metric] = temp_df
        df_aq10_grouped_boxplot[f'{date_range}_{percentile}'] = df_metrics
            
# Valid HUC2 gages
df_huc2_boxplot = {}
# Loop through date ranges and percentiles
for date_range in date_ranges:
    for percentile in percentiles:
        df_huc2 = {}
        # Loop through HUC2 codes
        for huc2 in huc2_codes:
            temp_df = dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['huc2_code'] == huc2]
            df_metrics = {}
            # Loop through metrics
            for metric in metric_list:
                metrics_desc = temp_df[metric].describe()
                #print(metrics_desc)
                df_metrics[metric] = metrics_desc
            df_huc2[huc2] = df_metrics
        df_huc2_boxplot[f'{date_range}_{percentile}'] = df_huc2

        
# Valid aquifer gages
df_aq_boxplot = {}
# Loop through date ranges and percentiles
for date_range in date_ranges:
    for percentile in percentiles:
        df_aq = {}
        # Loop through aquifer codes
        for aq in aq_codes_25:
            temp_df = dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['within_aq'] == aq_names_25[aq]]
            df_metrics = {}
            # Loop through metrics
            for metric in metric_list:
                metrics_desc = temp_df[metric].describe()
                df_metrics[metric] = metrics_desc
            df_aq[aq] = df_metrics
        df_aq_boxplot[f'{date_range}_{percentile}'] = df_aq

# Aquifer outlet gages
df_aq_outlet_boxplot = {}
# Loop through date ranges and percentiles
for date_range in date_ranges:
    for percentile in percentiles:
        df_aq = {}
        # Loop through aquifer codes
        for aq in aq_codes_25:
            temp_df = dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['outlet_gage'] == aq_names_25[aq]]
            df_metrics = {}
            # Loop through metrics
            for metric in metric_list:
                metrics_desc = temp_df[metric].describe()
                df_metrics[metric] = metrics_desc
            df_aq[aq] = df_metrics
        df_aq_outlet_boxplot[f'{date_range}_{percentile}'] = df_aq

# Valid region gages
df_region_boxplot = {}
# Loop through date ranges and percentiles
for date_range in date_ranges:
    for percentile in percentiles:
        df_region = {}
        # Loop through aquifer codes
        for region in unique_region_list:
            temp_df = dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['region'] == region]
            df_metrics = {}
            # Loop through metrics
            for metric in metric_list:
                metrics_desc = temp_df[metric].describe()
                df_metrics[metric] = metrics_desc
            df_region[region] = df_metrics
        df_region_boxplot[f'{date_range}_{percentile}'] = df_region

In [131]:
#df_aq_outlet_boxplot

In [114]:
#df_huc2_boxplot
def count_gages(df):
    summary = pd.DataFrame()
    if df == df_gages_boxplot:
        for date_range_perc, metrics_dict in df.items():
            for metric, values in metrics_dict.items():
                # Select the mean and standard deviation for the metric values
                count_val = values['count']
            summary.at[metric, date_range_perc] = f"{count_val}"
    elif (df == df_huc2_boxplot or df == df_region_boxplot):
        for date_range_perc, metrics_dict in df.items():
        # Handle the additional level in df_huc2_boxplot
            for inner_key, metrics in metrics_dict.items():
                for metric, values in metrics.items():
                    # Select the mean and standard deviation for the metric values
                    count_val = values['count']
                    
                    # Store the mean +/- std dev result in the summary table
                    summary.at[metric, f'{date_range_perc}_{inner_key}'] = f"{count_val}"
        pd.set_option('display.max_columns', None)
        print(summary)
    return summary

#count_gages(df_gages_boxplot)
#count_gages(df_huc2_boxplot)
#count_gages(df_region_boxplot)

# Mean +/- Std Tables

In [180]:
def create_mean_std_table(df):
    name = 'aq_outlet'
    # if df == df_gages_boxplot:
    #     name = 'valid_gages'
    # elif df == df_huc2_boxplot:
    #     name = 'huc2'
    # elif df == df_region_boxplot:
    #     name = 'region'
    # elif df == df_aq_boxplot:
    #     name = 'aq'
    # #else: 
    # elif df == df_aq_outlet_boxplot:
    #     name = 'aq_outlet'
    summary = pd.DataFrame(index=metric_list)
    
    # # Iterate through each date range/percentile combination in the dictionary
    # if df == df_gages_boxplot:
    #     for date_range_perc, metrics_dict in df.items():
    #         for metric, values in metrics_dict.items():
    #             # Select the mean and standard deviation for the metric values
    #             mean_val = values['mean']
    #             std_dev = values['std']

    #             # Format the mean and standard deviation based on the metric
    #             if metric == 'annual_hmf':
    #                 mean_val_formatted = f"{mean_val:.2f}"
    #                 std_dev_formatted = f"{std_dev:.2f}"
    #             elif metric == 'event_hmf':
    #                 mean_val_formatted = f"{mean_val:.3f}"
    #                 std_dev_formatted = f"{std_dev:.3f}"
    #             else:
    #                 mean_val_formatted = f"{mean_val:.0f}"
    #                 std_dev_formatted = f"{std_dev:.0f}"
                
    #             # Store the mean +/- std dev result in the summary table
    #             summary.at[metric, date_range_perc] = f"{mean_val_formatted} ± {std_dev_formatted}"
    #     file_name = f'mean_std_boxplot_{name}.xlsx'
    #     summary.to_excel('Tables/mean_std_tables/'+file_name)

    # elif (df == df_huc2_boxplot or df == df_region_boxplot or df == df_aq_boxplot or df_aq_outlet_boxplot):
    #     for date_range_perc, metrics_dict in df.items():
    #     # Handle the additional level in df_huc2_boxplot
    #         for inner_key, metrics in metrics_dict.items():
    #             for metric, values in metrics.items():
    #                 # Select the mean and standard deviation for the metric values
    #                 mean_val = values['mean']
    #                 std_dev = values['std']
    
    #                 # Format the mean and standard deviation based on the metric
    #                 if metric == 'annual_hmf':
    #                     mean_val_formatted = f"{mean_val:.2f}"
    #                     std_dev_formatted = f"{std_dev:.2f}"
    #                 elif metric == 'event_hmf':
    #                     mean_val_formatted = f"{mean_val:.3f}"
    #                     std_dev_formatted = f"{std_dev:.3f}"
    #                 else:
    #                     mean_val_formatted = f"{mean_val:.0f}"
    #                     std_dev_formatted = f"{std_dev:.0f}"
                    
    #                 # Store the mean +/- std dev result in the summary table
    #                 summary.at[metric, f'{date_range_perc}_{inner_key}'] = f"{mean_val_formatted} ± {std_dev_formatted}"

    for date_range_perc, metrics_dict in df.items():
    # Handle the additional level in df_huc2_boxplot
        for inner_key, metrics in metrics_dict.items():
            for metric, values in metrics.items():
                # Select the mean and standard deviation for the metric values
                mean_val = values['mean']
                std_dev = values['std']

                # Format the mean and standard deviation based on the metric
                if metric == 'annual_hmf':
                    mean_val_formatted = f"{mean_val:.2f}"
                    std_dev_formatted = f"{std_dev:.2f}"
                elif metric == 'event_hmf':
                    mean_val_formatted = f"{mean_val:.3f}"
                    std_dev_formatted = f"{std_dev:.3f}"
                else:
                    mean_val_formatted = f"{mean_val:.0f}"
                    std_dev_formatted = f"{std_dev:.0f}"
                
                # Store the mean +/- std dev result in the summary table
                summary.at[metric, f'{date_range_perc}_{inner_key}'] = f"{mean_val_formatted} ± {std_dev_formatted}"
    
    file_name = f'mean_std_boxplot_{name}.xlsx'
    summary.to_excel('Tables/mean_std_tables/'+file_name)
    return summary

# Display the summary table
#create_mean_std_table(df_gages_boxplot)
#create_mean_std_table(df_huc2_boxplot)
#create_mean_std_table(df_region_boxplot)
#create_mean_std_table(df_aq_boxplot)
create_mean_std_table(df_aq_outlet_boxplot)

,30_90_hp,30_90_mr,30_90_cv,30_90_br,30_90_fl,30_90_sr,30_90_cl,30_90_cc,30_90_pn,30_90_na,30_90_cp,30_90_rg,30_90_me,30_90_co,30_90_sc,30_90_bi,30_90_et,30_90_rm,30_90_pb,30_90_sa,30_90_vr,30_90_sd,30_90_pnb,30_90_wl,30_90_lc,30_95_hp,30_95_mr,30_95_cv,30_95_br,30_95_fl,30_95_sr,30_95_cl,30_95_cc,30_95_pn,30_95_na,30_95_cp,30_95_rg,30_95_me,30_95_co,30_95_sc,30_95_bi,30_95_et,30_95_rm,30_95_pb,30_95_sa,30_95_vr,30_95_sd,30_95_pnb,30_95_wl,30_95_lc,50_90_hp,50_90_mr,50_90_cv,50_90_br,50_90_fl,50_90_sr,50_90_cl,50_90_cc,50_90_pn,50_90_na,50_90_cp,50_90_rg,50_90_me,50_90_co,50_90_sc,50_90_bi,50_90_et,50_90_rm,50_90_pb,50_90_sa,50_90_vr,50_90_sd,50_90_pnb,50_90_wl,50_90_lc,50_95_hp,50_95_mr,50_95_cv,50_95_br,50_95_fl,50_95_sr,50_95_cl,50_95_cc,50_95_pn,50_95_na,50_95_cp,50_95_rg,50_95_me,50_95_co,50_95_sc,50_95_bi,50_95_et,50_95_rm,50_95_pb,50_95_sa,50_95_vr,50_95_sd,50_95_pnb,50_95_wl,50_95_lc
annual_hmf,0.18 ± 0.29,0.51 ± 0.65,2.26 ± 0.53,0.32 ± 0.33,0.90 ± 0.96,0.32 ± 0.29,1.01 ± 0.91,0.57 ± 0.83,1.52 ± 1.24,0.78 ± 1.18,1.24 ± 1.76,0.20 ± nan,0.47 ± 0.42,1.01 ± 1.06,1.44 ± 1.01,0.02 ± 0.01,1.02 ± 0.84,0.95 ± 0.92,0.54 ± 0.28,0.82 ± 0.87,1.09 ± 1.46,1.51 ± 1.29,1.61 ± 0.94,nan ± nan,0.34 ± 1.11,0.13 ± 0.20,0.33 ± 0.47,1.01 ± 0.37,0.31 ± 0.33,0.59 ± 0.60,0.20 ± 0.19,0.59 ± 0.43,0.38 ± 0.52,0.84 ± 0.55,0.47 ± 0.67,0.71 ± 1.03,0.10 ± nan,0.30 ± 0.27,0.62 ± 0.70,0.87 ± 0.49,0.02 ± 0.01,0.69 ± 0.48,0.52 ± 0.48,0.37 ± 0.21,0.51 ± 0.58,0.62 ± 0.83,0.92 ± 0.74,0.93 ± 0.56,nan ± nan,0.25 ± 0.83,0.18 ± 0.28,0.40 ± 0.19,2.34 ± 0.35,0.92 ± 0.75,0.81 ± 0.79,0.32 ± 0.25,0.99 ± 0.93,0.70 ± 0.99,1.52 ± 1.23,0.79 ± 1.23,1.25 ± 1.71,0.21 ± nan,0.48 ± 0.39,1.29 ± 1.17,1.77 ± 1.31,0.04 ± 0.01,0.97 ± 0.82,1.00 ± 0.99,0.55 ± 0.31,0.62 ± 0.65,1.13 ± 1.51,1.53 ± 1.31,2.30 ± nan,nan ± nan,0.21 ± 0.61,0.13 ± 0.19,0.22 ± 0.12,1.11 ± 0.55,0.89 ± 0.91,0.50 ± 0.45,0.16 ± 0.11,0.58 ± 0.44,0.46 ± 0.59,0.81 ± 0.53,0.48 ± 0.70,0.84 ± 1.30,0.12 ± nan,0.31 ± 0.25,0.78 ± 0.73,0.94 ± 0.60,0.02 ± 0.01,0.63 ± 0.42,0.53 ± 0.47,0.35 ± 0.20,0.36 ± 0.37,0.64 ± 0.85,0.86 ± 0.71,1.37 ± nan,nan ± nan,0.17 ± 0.55
annual_duration,39 ± 4,40 ± 4,73 ± 26,66 ± 35,44 ± 6,59 ± 18,40 ± 4,40 ± 5,38 ± 2,38 ± 3,41 ± 4,52 ± nan,37 ± 2,40 ± 4,40 ± 5,65 ± 34,45 ± 5,48 ± 11,38 ± 2,48 ± 9,37 ± 2,38 ± 2,58 ± 22,nan ± nan,44 ± 11,22 ± 5,22 ± 5,52 ± 23,47 ± 24,27 ± 7,41 ± 9,23 ± 4,22 ± 4,20 ± 1,21 ± 4,24 ± 4,34 ± nan,20 ± 1,24 ± 4,25 ± 5,39 ± 16,28 ± 5,28 ± 4,21 ± 1,29 ± 4,19 ± 1,23 ± 5,35 ± 11,nan ± nan,25 ± 9,38 ± 3,38 ± 3,73 ± 25,106 ± 29,44 ± 7,60 ± 11,40 ± 4,41 ± 4,38 ± 1,38 ± 3,41 ± 4,57 ± nan,38 ± 1,42 ± 4,41 ± 5,49 ± 14,44 ± 4,48 ± 10,38 ± 1,45 ± 7,37 ± 1,40 ± 3,44 ± nan,nan ± nan,43 ± 8,21 ± 3,20 ± 2,52 ± 26,101 ± 41,26 ± 7,36 ± 5,22 ± 3,23 ± 5,19 ± 1,21 ± 3,25 ± 5,39 ± nan,20 ± 2,25 ± 4,25 ± 4,29 ± 12,27 ± 4,28 ± 4,21 ± 1,27 ± 2,19 ± 1,23 ± 4,28 ± nan,nan ± nan,26 ± 8
event_duration,6 ± 2,11 ± 9,37 ± 22,28 ± 27,13 ± 12,27 ± 16,8 ± 3,9 ± 6,6 ± 3,7 ± 3,11 ± 4,34 ± nan,7 ± 3,11 ± 7,6 ± 2,14 ± 7,9 ± 3,32 ± 4,5 ± 3,14 ± 8,4 ± 1,9 ± 5,25 ± 2,nan ± nan,12 ± 8,4 ± 1,8 ± 6,24 ± 14,28 ± 28,10 ± 8,18 ± 9,6 ± 3,5 ± 3,4 ± 2,5 ± 3,7 ± 3,21 ± nan,5 ± 2,9 ± 5,5 ± 2,12 ± 6,7 ± 2,18 ± 2,4 ± 2,10 ± 4,3 ± 1,8 ± 4,20 ± 2,nan ± nan,9 ± 7,6 ± 2,7 ± 4,35 ± 19,70 ± 20,14 ± 14,23 ± 12,8 ± 3,10 ± 6,6 ± 3,7 ± 4,10 ± 3,34 ± nan,7 ± 3,14 ± 5,7 ± 2,12 ± 10,9 ± 3,28 ± 1,5 ± 3,14 ± 7,4 ± 1,10 ± 5,26 ± nan,nan ± nan,11 ± 7,4 ± 1,5 ± 3,24 ± 17,47 ± 5,11 ± 9,13 ± 4,6 ± 3,6 ± 3,4 ± 2,5 ± 3,7 ± 2,21 ± nan,5 ± 2,11 ± 4,6 ± 1,8 ± 6,7 ± 2,18 ± 4,4 ± 2,11 ± 4,3 ± 1,8 ± 4,18 ± nan,nan ± nan,8 ± 6
event_hmf,0.025 ± 0.041,0.232 ± 0.485,1.132 ± 0.139,0.152 ± 0.191,0.249 ± 0.258,0.151 ± 0.127,0.216 ± 0.208,0.104 ± 0.137,0.334 ± 0.442,0.124 ± 0.191,0.366 ± 0.516,0.136 ± nan,0.090 ± 0.084,0.365 ± 0.504,0.197 ± 0.152,0.006 ± 0.005,0.312 ± 0.401,0.684 ± 0.672,0.066 ± 0.023,0.277 ± 0.338,0.145 ± 0.219,0.462 ± 0.605,0.866 ± 0.753,nan ± nan,0.124 ± 0.414,0.025 ± 0.040,0.193 ± 0.398,0.510 ± 0.311,0.197 ± 0.292,0.200 ± 0.179,0.092 ± 0.096

## Save boxplot data as separate tab for each region

In [241]:
data = df_huc2_boxplot
for time_key, sites_dict in data.items():
    site_df_dict = {}
    
    # Initialize an empty DataFrame for each site within the current `time_key`
    for site_key, metrics_dict in sites_dict.items():
        site_df_dict[f'{time_key}_site_{site_key}'] = pd.DataFrame()

        # For each metric, convert it into a Series and add it as a column to the DataFrame
        for metric, stats in metrics_dict.items():
            metric_series = pd.Series(stats, name=metric)
            site_df_dict[f'{time_key}_site_{site_key}'][metric] = metric_series

    # Save each `time_key` to a separate Excel file
    excel_filename = f'boxplot_huc2_metrics_{time_key}.xlsx'
    with pd.ExcelWriter(f'Tables/{excel_filename}') as writer:
        for sheet_name, df in site_df_dict.items():
            df.to_excel(writer, sheet_name=sheet_name)

In [247]:
data = df_aq_boxplot
for time_key, sites_dict in data.items():
    site_df_dict = {}
    
    # Initialize an empty DataFrame for each site within the current `time_key`
    for site_key, metrics_dict in sites_dict.items():
        site_df_dict[f'{time_key}_site_{site_key}'] = pd.DataFrame()

        # For each metric, convert it into a Series and add it as a column to the DataFrame
        for metric, stats in metrics_dict.items():
            metric_series = pd.Series(stats, name=metric)
            site_df_dict[f'{time_key}_site_{site_key}'][metric] = metric_series

    # Save each `time_key` to a separate Excel file
    excel_filename = f'boxplot_aq25_metrics_{time_key}.xlsx'
    with pd.ExcelWriter(f'Tables/{excel_filename}') as writer:
        for sheet_name, df in site_df_dict.items():
            df.to_excel(writer, sheet_name=sheet_name)

In [273]:
data = df_region_boxplot
for time_key, sites_dict in data.items():
    site_df_dict = {}
    
    # Initialize an empty DataFrame for each site within the current `time_key`
    for site_key, metrics_dict in sites_dict.items():
        site_df_dict[f'{time_key}_site_{site_key}'] = pd.DataFrame()

        # For each metric, convert it into a Series and add it as a column to the DataFrame
        for metric, stats in metrics_dict.items():
            metric_series = pd.Series(stats, name=metric)
            site_df_dict[f'{time_key}_site_{site_key}'][metric] = metric_series

    # Save each `time_key` to a separate Excel file
    excel_filename = f'boxplot_region_metrics_{time_key}.xlsx'
    with pd.ExcelWriter(f'Tables/{excel_filename}') as writer:
        for sheet_name, df in site_df_dict.items():
            df.to_excel(writer, sheet_name=sheet_name)

In [42]:
# data = df_gages_boxplot
# name = 'valid_gages'

data = df_aq25_grouped_boxplot
name = 'aq25_all_gages'

#data = df_aq10_grouped_boxplot
#name = 'aq10_all_gages'

dfs = {}

# Loop over the outer dictionary and convert each nested dictionary into a DataFrame
for key, metrics in data.items():
    # Convert the inner dictionary to a DataFrame
    df = pd.DataFrame(metrics)
    dfs[key] = df

for date_range in date_ranges:
    for percentile in percentiles:
        file_name = f'boxplot_{name}_{date_range}_{percentile}.xlsx'
        dfs[f'{date_range}_{percentile}'].to_excel('Tables/'+file_name)

KeyError: '30_90'

In [111]:
date_range = '30'
percentile = '90'
temp_df = dfs_valid[f'{date_range}_{percentile}'][dfs_valid[f'{date_range}_{percentile}']['within_aq'].isin(aq_names_25_list)]
temp_df['annual_duration'].describe()

count    1690.000000
mean       40.115056
std         8.903198
min         6.629630
25%        36.464881
50%        36.627315
75%        40.555556
max       156.428571
Name: annual_duration, dtype: float64